In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time

In [4]:
pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [5]:
import Bio
from Bio import SeqIO

In [6]:
list_chromosomes=[]
for seq_record in SeqIO.parse("GCF_000002765.5_GCA_000002765_genomic.fna", "fasta"):
    print(seq_record.id)
    list_chromosomes.append(str(seq_record.seq))
    print(len(seq_record))

NC_004325.2
640851
NC_037280.1
947102
NC_000521.4
1067971
NC_004318.2
1200490
NC_004326.2
1343557
NC_004327.3
1418242
NC_004328.3
1445207
NC_004329.3
1472805
NC_004330.2
1541735
NC_037281.1
1687656
NC_037282.1
2038340
NC_037284.1
2271494
NC_004331.3
2925236
NC_037283.1
3291936
NC_036769.1
34250


In [7]:
def counting_sort(array,ind):
    list_index = [0]*len(array)
    for i in range(len(array)):
        list_index[i]=array[i][ind]
    
    x = max(list_index)
    count_arr= [0]*(x+1)
    sorted_arr=[0]*len(array)
    j=0
    
    for i in range(0, len(array)):
        index = array[i][ind]
        count_arr[index] += 1  
    
    for i in range(1, len(count_arr)):
        count_arr[i] += count_arr[i - 1]
    
    i = len(array) - 1
    
    while i >= 0:
        index = array[i][ind]
        sorted_arr[count_arr[index] - 1] = array[i]
        count_arr[index] -= 1
        i -= 1
        #print("sort", sorted_arr)
        
    array = sorted_arr
    return array

def RadixSort(array):
    x = len(array[0])
    ind=x-1
    sort=array
    while ind>=0 :
        sort=counting_sort(sort,ind)
        ind -=1
    return sort

In [8]:
def index_merge(input_sequence, index_0, index_12, dict_index12):
    """
    Merge index_0 and index_12 indexes
    
    Args:
        input_sequence (list of int): the input string that has been converted to int
        index_0 (list of int) : list of indexes in which mod by 3 gives 0, sorted following R0 order
        index_12 (list of int) : list of indexes in which mod by 3 gives 1 or 2, sorted following R_sorted order
        dict_index (dict : map int over int) : indexes of index_12 as keys and index of the index_12 as values
        
        
    Return:
        list of int: the merging of index_0 and index_12
    """
        
    res=[]
    i=0
    j=0
    while i < len(index_0) and j < len(index_12):
        a = index_0[i]
        b = index_12[j]
        if input_sequence[a] < input_sequence[b]:
            res.append(a)
            i=i+1
        elif input_sequence[a] > input_sequence[b]:
            res.append(b)
            j=j+1
        else:
            if b%3==1:
                c = dict_index12[a+1]
                d = dict_index12[b+1]
                if c<d:
                    res.append(a)
                    i=i+1
                elif d<c:
                    res.append(b)
                    j=j+1
    
            if b%3==2:
                if input_sequence[a+1] < input_sequence[b+1]:
                    res.append(a)
                    i=i+1
                elif input_sequence[a+1] > input_sequence[b+1]:
                    res.append(b)
                    j=j+1
                if input_sequence[a+1] == input_sequence[b+1]:
                    c = dict_index12[a+2]
                    d = dict_index12[b+2]
                    if c<d:
                        res.append(a)
                        i=i+1
                    elif d<c:
                        res.append(b)
                        j=j+1
    if i == len(index_0):
        for index in index_12[j:]:
            res.append(index)
    elif j==len(index_12):
        for index in index_0[i:]:
            res.append(index)
    return res 

In [9]:
def dc3(input_string):
    """
    Compute the suffix array using the DC3 algorithm.
    
    Args:
        input_string (str): the input string for which to compute the suffix array.
        input_string (list of int) : the input string is a list of int if dc3 has been called recursively
    Return:
        list of int: The suffix array.
    """

    # Step 0 : convert the input string to a list of integers if the input string is a string
    if type(input_string)==str:
        input_sequence = [ord(char) for char in input_string]
    else:
        input_sequence = input_string.copy()
    input_sequence.extend((0,0,0)) #add centinels
    
    # Step 1 : Split the sequence into three subsequences based on mod by 3
    input_seq_len=len(input_sequence)
    P0 = [i for i in range(input_seq_len-1) if ( i%3==0 and i+2<input_seq_len)]
    P1 = [i for i in range(input_seq_len-1) if ( i%3==1 and i+2<input_seq_len)]
    P2 = [i for i in range(input_seq_len-1) if ( i%3==2 and i+2<input_seq_len)]
    # concatenation of P1 and P2
    P12 = P1 + P2
    
    # create triplets (R12) of numbers starting from indexes P12 and sort them
    R12 = [(input_sequence[i], input_sequence[i+1], input_sequence[i+2]) for i in P12]
    R_sorted = RadixSort(R12)

    # Create a dict with triplets as keys and their order in R_sorted as value.
    dict_Rsorted_order = {}
    duplicate = False
    order = 1
    for triplet in R_sorted:
        if triplet not in dict_Rsorted_order:
            dict_Rsorted_order[triplet] = order
            order += 1
        else:
            duplicate = True


    # Create a dictionary with triplets as keys and a subdictionnary of their indexes (index of the first number of the triplet) as values.
    # If no duplicate, the subdictionnary is { 0 : index }
    # If duplicate, the subdictionnary is { 0 : index1, 1 : index2, ...}
    dict_triplet_index={}
    for i in range(len(P12)):
        if R12[i] not in dict_triplet_index:
            dict_triplet_index[R12[i]]={0:P12[i]}
        else:
            length=len(dict_triplet_index[R12[i]])
            dict_triplet_index[R12[i]][length]=P12[i]

    
    # Create the index_12 containing the original indexes of each triplet in sorted order
    index_12 = []
    dict_index12={} #keep in memory the index_12 as key and the index of the index_12 as value, it is for the R0 computation to make it faster
    i=0
    for triplet in R_sorted:
        current_key, current_index = dict_triplet_index[triplet].popitem()
        index_12.append(current_index)
        dict_index12[current_index] = i
        i += 1
    
    # If there are duplicate triplets, process T' and repeat recursively
    if duplicate:
        T_prime = []
        for elem in R12:
            T_prime.append(dict_Rsorted_order[elem])
        index_12=[]
        i=0
        dict_index12={}
        index_prime = dc3(T_prime)
        for ind in index_prime:
            index_12.append(P12[ind])
            dict_index12[P12[ind]]=i
            i+=1


    # Step 2 : Create R0 and sort it.
    R0 = []
    for num in P0:
        if num < len(input_string): #if num is a centinel, so if num=len(S), there is no triplet at index num+1
            R0.append([input_sequence[num],dict_index12[num+1]])       
    R0_sorted = RadixSort(R0) 

    # Create the index_0 list 
    index_0=[]
    for elem in R0_sorted:
        index_0.append(index_12[elem[1]]-1)


    # Step 3 : merge index_0 and index_12

    index_merged = index_merge(input_sequence,index_0,index_12, dict_index12)

    #Remove centinel index. Index of centinel is always len(input_string)
    new_index = index_merged
    if len(input_string) in new_index:
        new_index.remove(len(input_string))

        
    return new_index

In [10]:
st=time.time()
i = 1
for chrom in list_chromosomes:
    dc3(chrom)
    print("Chromosome", i, "done.")
    i+=1
ed=time.time()
print(ed-st)

Chromosome 1 done.
Chromosome 2 done.
Chromosome 3 done.
Chromosome 4 done.
Chromosome 5 done.
Chromosome 6 done.
Chromosome 7 done.
Chromosome 8 done.
Chromosome 9 done.
Chromosome 10 done.
Chromosome 11 done.
Chromosome 12 done.
Chromosome 13 done.
Chromosome 14 done.
Chromosome 15 done.
410.99075269699097
